In [354]:
library(tidyverse)
library(lubridate)
library(forecast) # Para modelos AR

In [491]:
df<-read_csv('df.csv',col_names = TRUE,cols(
  Ano = col_double(),
  Mes = col_double(),
  Ano_Mes = col_character(),
  Fecha = col_date(format = ""),
  Precios_Mensuales = col_double(),
  Aportes_Energia_GWh = col_double(),
  Volumen_Util_Diario_Energia_GWh = col_double(),
  Demanda_Energia_SIN_GWh = col_double(),
  ONI = col_double(),
  CEN_Total = col_double(),
  CEN_Hidro = col_double(),
  CEN_Termica = col_double()
))

# Creacion de funciones

In [535]:
oni<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    
    n_pronosticos<-df%>%filter(Fecha>=start_at)%>%select(ONI)%>%nrow
    
    
    while(any(is.na(df$ONI))){
        fila<-sum(complete.cases(df$ONI))+1
        m<-auto.arima(unlist(unname(df[1:fila-1,'ONI'])),max.q = 0,max.Q = 0,max.d = 0,max.D = 0)
        y_est<-round(unname(unlist(forecast(m,h = 1)['mean'])),1)
        df[fila,'ONI']<-y_est
    }
    #df$ONI[df$Fecha>=as.Date(start_at)]<-round(runif(n_pronosticos,-0.5,0.5),1)
    
    return(df)
}

In [538]:
aportes<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Aportes_Energia_GWh)%>%nrow
    df$Aportes_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Aportes_Energia_GWh,na.rm = TRUE),max(df$Aportes_Energia_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [539]:
reservas<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Volumen_Util_Diario_Energia_GWh)%>%nrow
    df$Volumen_Util_Diario_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),max(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [540]:
precios<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Precios_Mensuales)%>%nrow
    df$Precios_Mensuales[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Precios_Mensuales,na.rm = TRUE),max(df$Precios_Mensuales,na.rm = TRUE)),3)
    
    return(df)
}

In [541]:
demanda<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Demanda_Energia_SIN_GWh)%>%nrow
    df$Demanda_Energia_SIN_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Demanda_Energia_SIN_GWh,na.rm = TRUE),max(df$Demanda_Energia_SIN_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [542]:
columnas_calculadas<-function(df){
    #debo montar dem_CEN
    df$Dem_CEN<-round(df$Demanda_Energia_SIN_GWh/(as.vector(days_in_month(df$Fecha))*df$CEN_Total*24),4)
    #debo montar Term_Hidro
    df$Termica_Hidraulica<-round(df$CEN_Termica/df$CEN_Hidro,4)
    
    return(df)
}

In [543]:
normalizar_hidrologia<-function(df){
    df<-df%>%
        left_join(
                df%>%arrange(Mes)%>%group_by(Mes)%>%summarize(mean_aportes = mean(Aportes_Energia_GWh,na.rm = TRUE), 
                                                              mean_reservas = mean(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),
                                                              sd_aportes = sd(Aportes_Energia_GWh, na.rm = TRUE),
                                                              sd_reservas = sd(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),by='Mes')%>%
    mutate(
        Aportes_Normalizados = (Aportes_Energia_GWh-mean_aportes)/sd_aportes,
        Reservas_Normalizadas = (Volumen_Util_Diario_Energia_GWh-mean_reservas)/sd_reservas)
    
    return(df)
}

In [544]:
head(df)

Ano,Mes,Ano_Mes,Fecha,Precios_Mensuales,Aportes_Energia_GWh,Volumen_Util_Diario_Energia_GWh,Demanda_Energia_SIN_GWh,ONI,CEN_Total,CEN_Hidro,CEN_Termica
2000,1,2000-01,2000-01-01,36.77878,1900.660,10115.866,3354.237,-1.7,11.88597,7.97497,3.897
2000,2,2000-02,2000-02-01,40.26148,1993.636,9705.065,3278.267,-1.4,11.88597,7.97497,3.897
2000,3,2000-03,2000-03-01,37.63730,2253.361,9222.966,3495.203,-1.1,11.96897,8.05797,3.897
2000,4,2000-04,2000-04-01,44.29806,2429.651,8945.684,3298.386,-0.8,12.05397,8.14297,3.897
2000,5,2000-05,2000-05-01,37.29112,4817.901,10131.148,3502.127,-0.7,12.29927,8.23427,4.051
2000,6,2000-06,2000-06-01,39.99320,4889.563,11020.110,3367.886,-0.6,12.44327,8.23427,4.195


# Probar funciones

In [545]:
df<-precios(df,start_at = '2019-06-01')
df<-oni(df,start_at = '2019-06-01')
df<-aportes(df,start_at = '2019-06-01')
df<-reservas(df,start_at = '2019-06-01')
df<-demanda(df,start_at = '2019-06-01')
df<-columnas_calculadas(df)
df<-normalizar_hidrologia(df)
str(df)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	257 obs. of  20 variables:
 $ Ano                            : num  2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 ...
 $ Mes                            : num  1 2 3 4 5 6 7 8 9 10 ...
 $ Ano_Mes                        : chr  "2000-01" "2000-02" "2000-03" "2000-04" ...
 $ Fecha                          : Date, format: "2000-01-01" "2000-02-01" ...
 $ Precios_Mensuales              : num  36.8 40.3 37.6 44.3 37.3 ...
 $ Aportes_Energia_GWh            : num  1901 1994 2253 2430 4818 ...
 $ Volumen_Util_Diario_Energia_GWh: num  10116 9705 9223 8946 10131 ...
 $ Demanda_Energia_SIN_GWh        : num  3354 3278 3495 3298 3502 ...
 $ ONI                            : num  -1.7 -1.4 -1.1 -0.8 -0.7 -0.6 -0.6 -0.5 -0.5 -0.6 ...
 $ CEN_Total                      : num  11.9 11.9 12 12.1 12.3 ...
 $ CEN_Hidro                      : num  7.97 7.97 8.06 8.14 8.23 ...
 $ CEN_Termica                    : num  3.9 3.9 3.9 3.9 4.05 ...
 $ Dem_CEN              

In [546]:
tail(df)

Ano,Mes,Ano_Mes,Fecha,Precios_Mensuales,Aportes_Energia_GWh,Volumen_Util_Diario_Energia_GWh,Demanda_Energia_SIN_GWh,ONI,CEN_Total,CEN_Hidro,CEN_Termica,Dem_CEN,Termica_Hidraulica,mean_aportes,mean_reservas,sd_aportes,sd_reservas,Aportes_Normalizados,Reservas_Normalizadas
2020,12,2020-12,2020-12-01,892.941,3266.028,7725.894,5473.457,0.8,13.15767,8.67967,4.464,0.5591,0.5143,3549.547,11519.177,1247.7605,1347.602,-0.227222,-2.8148399
2021,1,2021-01,2021-01-01,304.793,9520.223,9874.873,3915.356,0.8,13.15767,8.67967,4.464,0.4000,0.5143,2918.066,10335.935,1944.3293,1012.034,3.395596,-0.4555798
2021,2,2021-02,2021-02-01,224.495,2714.175,11175.058,5746.467,0.8,13.15867,8.68067,4.464,0.6499,0.5142,2041.531,9246.496,582.2503,1411.523,1.155248,1.3662986
2021,3,2021-03,2021-03-01,275.863,5232.291,10565.249,3986.024,0.8,13.15867,8.68067,4.464,0.4072,0.5142,3241.081,8343.221,1521.0384,1716.493,1.309112,1.2945166
2021,4,2021-04,2021-04-01,584.456,7624.242,9759.575,4479.136,0.8,13.15867,8.68067,4.464,0.4728,0.5142,4826.470,8617.229,2132.1928,1997.070,1.312157,0.5720111
2021,5,2021-05,2021-05-01,137.428,9823.246,11897.966,4041.771,0.8,13.16023,8.68223,4.464,0.4128,0.5142,6240.113,9733.016,1844.5686,1842.675,1.942532,1.1748956


In [233]:
generar_archivos<-function(df, start_at, n_iter, folder_location){
    
    
    if(dir.exists(folder_location)){
        while(file.remove(list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1])==TRUE){
            list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1]
        }

        fechaini<-as.Date(start_at)

        for( i in seq(n_iter)){

            #Normalizar aportes
            #normalizar reservas
            #Agregar variables calculadas
            df<-precios(df,fechaini)
            df<-oni(df,fechaini)
            df<-aportes(df,fechaini)
            df<-reservas(df,fechaini)
            df<-demanda(df,fechaini)
            df<-columnas_calculadas(df)

            #pronostico de variables calculadas
            write_csv(df, paste0(folder_location,str_pad(i, pad = 0, width = 5),'.csv'))
        }
        cat(paste0(n_iter, ' archivos generados en la ruta ',folder_location))
    } else{
        cat('Error: Directorio no existe en esta mquina')
    }

}

In [234]:
generar_archivos(df,start_at = '2019-06-01',n_iter = 20,folder_location = './Resultados/')

20 archivos generados en la ruta ./Resultados/

# Tareas

In [235]:
#para el oni es un AR qe estime el P optimo y genere hacia adelante una serie sintetica y llenar 2h
#hacer el arx pronosticar uno a uno y tiene como entrada el ONI y se devuelven las unidades 2h
#hacer arx con oni, aportes, relacion T/H y me devuelvo 2h
#un modelo simple de precio 2h